### 方案1：ListMemory

**文档预处理：**
1. PDF文档处理 
2. 文本分割
3. 摘要生成
4. 存入listMemory

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, ListMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

# 1. 文档处理
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
with open("project_docs.txt") as f:
    docs = [Document(page_content=text) for text in text_splitter.split_text(f.read())]

# 2. 生成摘要
llm = ChatOpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
summaries = summary_chain.run(docs)

# 3. 创建Agent并配置内存
knowledge_memory = ListMemory()
knowledge_memory.add(summaries)  # 添加生成的摘要

assistant = AssistantAgent(
    name="assistant",
    system_message="你是一个项目助手，当用户询问项目相关问题时，请参考以下知识库：\n{{知识库}}",
    memory=knowledge_memory,
    llm_config={"config_list": config_list}
)

user_proxy = UserProxyAgent(...)

### stuff chain，一次性生成摘要

In [7]:
import os
import asyncio
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from autogen_agentchat.ui import Console
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
import re
import nest_asyncio

# 允许嵌套的事件循环
nest_asyncio.apply()

# 加载环境变量
load_dotenv()

# 自定义章节分块器
class ChapterAwareSplitter:
    def __init__(self, chunk_size=2000):
        self.chunk_size = chunk_size
        self.chapter_pattern = re.compile(
            r'(^#+\s.+)|(^第[一二三四五六七八九十]+章\s.+)|(^\d+\.\d+\s.+)', 
            re.MULTILINE
        )

    def split_text(self, text):
        chapters = []
        buffer = ""
        
        # 使用finditer来获取所有的章节开头，这样可以更好地控制不打断章节
        for match in self.chapter_pattern.finditer(text):
            start_pos = match.start()
            
            if len(buffer) + start_pos > self.chunk_size and buffer:
                chapters.append(buffer)
                buffer = ""
                
            buffer += text[:start_pos]
            text = text[start_pos:]
            
            chapter_text = match.group()
            if len(buffer) + len(chapter_text) > self.chunk_size and buffer:
                chapters.append(buffer)
                buffer = ""
            buffer += chapter_text
        
        if text:  # 添加剩余的文本
            buffer += text
            
        if buffer:
            chapters.append(buffer)
            
        return chapters

# PDF处理器类
class PDFKnowledgeProcessor:
    def __init__(self, pdf_path, prompt_template):
        self.pdf_path = pdf_path
        self.prompt = PromptTemplate.from_template(prompt_template)
        self.llm = ChatOpenAI(
            temperature=0,
            model="gpt-4o-2024-08-06",
            base_url="https://api2.road2all.com/v1",
            api_key=os.getenv("OPENAI_API_KEY")
        )

    def process_pdf(self):
        from langchain_core.documents import Document
        # 1. 正确加载PDF内容
        try:
            loader = PyPDFLoader(self.pdf_path)
            pages = loader.load()
            full_text = "\n".join([page.page_content for page in pages])  # 直接提取文本
        except Exception as e:
            print(f"PDF加载失败: {str(e)}")
            return ""

        # 2. 章节分块
        splitter = ChapterAwareSplitter()
        chunks = splitter.split_text(full_text)
        print(f"成功分割为 {len(chunks)} 个章节块")

        # 将字符串块转换为Document对象
        documents = [
            Document(
                page_content=chunk,
                metadata={"source": self.pdf_path}  # 添加元数据
            ) for chunk in chunks
        ]
        
        # 3. 生成摘要
        try:
            chain = load_summarize_chain(
                self.llm,
                chain_type="map_reduce",
                map_prompt=self.prompt,
                combine_prompt=self.prompt,
                verbose=True
            )
            result = chain.invoke({"input_documents": documents})
            return result["output_text"]
        except Exception as e:
            print(f"摘要生成失败: {str(e)}")
            return ""

# 使用示例
async def main():
    # 配置参数
    PDF_PATH = "/home/www/AgentFlow/agentflow/docs/GalsimToolkit.pdf"
    PROMPT_TEMPLATE = """请用简洁的语言总结以下内容：
    
    {text}
    
    关键要点："""

    # 处理PDF
    processor = PDFKnowledgeProcessor(PDF_PATH, PROMPT_TEMPLATE)
    knowledge = processor.process_pdf()
    
    if knowledge:
        print("\n生成的知识库内容：\n", knowledge)
    else:
        print("知识库生成失败，请检查日志")
        return

    # 配置助手Agent
    memory = ListMemory()
    await memory.add(MemoryContent(
        content=knowledge,
        mime_type=MemoryMimeType.TEXT))

    model_client = OpenAIChatCompletionClient(
        model = "gpt-4o-2024-08-06",
        base_url = "https://api2.road2all.com/v1",
        api_key = os.getenv("OPENAI_API_KEY")
    )

    assistant_agent = AssistantAgent(
        name="project_assistant",
        system_message="你是一个项目助手，回答时请参考以下知识：\n{{知识库}}",
        model_client=model_client,
        memory=[memory]
    )

    stream = assistant_agent.run_stream(task="请根据文档说明项目使用的主要技术方法。")
    await Console(stream)


### refine chain，章节prompt+全文prompt

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from autogen_agentchat.ui import Console
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
import re
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

class ChapterAwareSplitter:
    def __init__(self, chunk_size=2000):
        self.chunk_size = chunk_size
        self.chapter_pattern = re.compile(
            r'(^#+\s.+)|(^第[一二三四五六七八九十]+章\s.+)|(^\d+\.\d+\s.+)', 
            re.MULTILINE
        )

    def split_text(self, text):
        chapters = []
        buffer = ""
        
        for match in self.chapter_pattern.finditer(text):
            start_pos = match.start()
            
            if len(buffer) + start_pos > self.chunk_size and buffer:
                chapters.append(buffer)
                buffer = ""
                
            buffer += text[:start_pos]
            text = text[start_pos:]
            
            chapter_text = match.group()
            if len(buffer) + len(chapter_text) > self.chunk_size and buffer:
                chapters.append(buffer)
                buffer = ""
            buffer += chapter_text
        
        if text:  
            buffer += text
            
        if buffer:
            chapters.append(buffer)
            
        return chapters

class PDFKnowledgeProcessor:
    def __init__(self, pdf_path, map_prompt, combine_prompt):
        self.pdf_path = pdf_path
        self.map_prompt = map_prompt
        self.combine_prompt = combine_prompt
        self.llm = ChatOpenAI(
            temperature=0,
            model="gpt-4o-2024-08-06",
            base_url="https://api2.road2all.com/v1",
            api_key=os.getenv("OPENAI_API_KEY")
        )

    def process_pdf(self):
        from langchain_core.documents import Document
        try:
            loader = PyPDFLoader(self.pdf_path)
            pages = loader.load()
            full_text = "\n".join([page.page_content for page in pages])
        except Exception as e:
            print(f"PDF加载失败: {str(e)}")
            return ""

        splitter = ChapterAwareSplitter()
        chunks = splitter.split_text(full_text)
        print(f"成功分割为 {len(chunks)} 个章节块")

        # 空文档过滤
        valid_chunks = []
        for chunk in chunks:
            if len(chunk.strip()) < 50:  # 过滤空内容
                continue
            if "未定义" in chunk or "待补充" in chunk:  # 过滤占位内容
                continue
            valid_chunks.append(chunk)
        
        if not valid_chunks:
            print("无有效内容需要处理")
            return ""

        documents = [
            Document(
                page_content=chunk,
                metadata={
                    "source": self.pdf_path,
                    "length": len(chunk)
                    }
            ) for chunk in valid_chunks
        ]

        try:
            chain = load_summarize_chain(
                self.llm,
                chain_type="refine",
                question_prompt=PromptTemplate.from_template(self.map_prompt),
                refine_prompt=PromptTemplate.from_template(self.combine_prompt),
                return_intermediate_steps=True,
                input_key="input_documents",
                output_key="output_text"
            )

            filtered_documents = []
            for doc in documents:
                if "无意义内容，无需总结" in doc.page_content:
                    continue
                filtered_documents.append(doc)
                # 调用链前，确保输入文档有效
            if not filtered_documents:
                print("没有有效文档需要处理")
                return ""

            # 调用链并捕获异常
            try:
                result = chain.invoke({"input_documents": filtered_documents})
                return result["output_text"]
            except Exception as e:
                print(f"链调用失败: {str(e)}")
                return ""
        except Exception as e:
            print(f"摘要生成失败: {str(e)}")
            return ""

async def main():
    PDF_PATH = "/home/www/AgentFlow/agentflow/docs/GalsimToolkit.pdf"
    map_prompt = """
    请仔细阅读以下文本内容，并回答以下问题（用中文，保持简洁）：
    如果内容中没有涉及以下任何一项，请直接回复“无意义内容，无需总结”：
    1. 技术方法/算法描述
    2. 代码模块/组件的结构说明
    3. 功能实现的关键步骤或流程
    4. 项目整体架构或作用说明

    如果内容有实际技术信息，请参考以下内容总结：
    1. "技术方法": "...",
    2. "模块结构": "...",
    3. "关键步骤": "...",
    4. "作用说明": "..."
    文本内容：
    {text}
    """

    combine_prompt = """
    请根据以下分块摘要，生成项目的技术概述文档。要求：
    1. **技术方法**：总结项目的核心技术、算法或框架。
    2. **架构设计**：描述代码的整体架构和模块间关系。
    3. **功能模块**：列举主要功能模块及其作用。
    4. **实现细节**：说明关键步骤或技术难点的解决方案。
    5. **应用场景**：项目适用的使用场景或目标用户。
    6. **优势特点**：项目的独特优势或创新点。

    请用结构化的方式呈现，分点说明，避免冗余。

    
    分块摘要：
    {text}
    """

    processor = PDFKnowledgeProcessor(
        pdf_path=PDF_PATH,
        map_prompt=map_prompt,
        combine_prompt=combine_prompt
    )
    knowledge = processor.process_pdf()
    
    if knowledge:
        print("\n生成的知识库内容：\n", knowledge)
    else:
        print("知识库生成失败，请检查日志")
        return

    memory = ListMemory()
    await memory.add(MemoryContent(
        content=knowledge,
        mime_type=MemoryMimeType.TEXT))

    model_client = OpenAIChatCompletionClient(
        model="gpt-4o-2024-08-06",
        base_url="https://api2.road2all.com/v1",
        api_key=os.getenv("OPENAI_API_KEY")
    )

    assistant_agent = AssistantAgent(
        name="project_assistant",
        system_message="你是一个项目助手，回答时请参考以下知识：\n{{知识库}}",
        model_client=model_client,
        memory=[memory]
    )

    stream = assistant_agent.run_stream(task="请根据文档总结项目的基本信息。")
    await Console(stream)

In [13]:
await main()

成功分割为 22 个章节块

生成的知识库内容：
 # 项目技术概述文档

## 1. 技术方法
- **核心技术**：项目使用了GAL SIM软件包来模拟天文图像中的噪声和图像处理。主要技术包括噪声相关函数的估计、噪声美白技术以及重卷积算法。
- **算法**：噪声美白技术通过添加额外的噪声来使图像中的噪声场变得近似不相关和稳定。重卷积算法用于处理低分辨率下的图像变换。
- **框架**：项目基于GAL SIM框架，提供了多种图像渲染方法，包括离散傅里叶变换（DFT）和光子射击。

## 2. 架构设计
- **整体架构**：项目代码分为多个模块，包括噪声处理模块、图像渲染模块、变换模块等。
- **模块间关系**：噪声处理模块负责估计和处理图像中的噪声，图像渲染模块负责生成模拟图像，变换模块处理图像的几何变换。

## 3. 功能模块
- **噪声处理模块**：估计和处理图像中的噪声，包括噪声相关函数的计算和噪声美白。
- **图像渲染模块**：使用DFT和光子射击方法生成模拟图像。
- **变换模块**：处理图像的几何变换，包括剪切和放大。
- **重卷积模块**：处理低分辨率图像的重卷积。

## 4. 实现细节
- **关键步骤**：噪声美白通过计算噪声的功率谱并添加额外的噪声来实现。重卷积算法通过模拟低分辨率下的图像变换来实现。
- **技术难点解决方案**：使用高精度的傅里叶变换和光子射击方法来确保图像渲染的准确性。通过参数调节来优化噪声处理和图像渲染的性能。

## 5. 应用场景
- **使用场景**：项目适用于天文图像的模拟和处理，特别是在弱透镜剪切和放大测量中。
- **目标用户**：天文学家、天文数据分析师以及需要高精度图像模拟的研究人员。

## 6. 优势特点
- **独特优势**：项目提供了多种图像渲染方法，能够处理复杂的噪声和图像变换。噪声美白技术能够显著降低噪声相关性。
- **创新点**：通过重卷积算法处理低分辨率图像变换，确保在不同分辨率下的图像处理精度。提供了灵活的参数调节机制以优化性能。
---------- user ----------
请根据文档总结项目的基本信息。
---------- project_assistant ----------
[MemoryContent(content='# 项目技术概述文档\n\

### langchain rag tools

In [ ]:
# 安装必要库
# pip install pyautogen langchain pypdf chromadb

import os
import getpass
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from autogen import AssistantAgent


# 1. 配置第三方API参数
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
BASE_URL = "https://api2.road2all.com/v1" 


# 2. 加载并处理PDF文档
def load_and_process_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(pages)
    return splits

# 3. 创建向量数据库
def create_vector_store(documents):
    # 配置自定义端点
    embeddings = OpenAIEmbeddings(
        openai_api_base=BASE_URL, 
        model="text-embedding-3-large", 
    )
    
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory="./langchain_vectors"
    )
    return vectorstore

# 4. 创建检索工具
def create_retrieval_tool(vectorstore):
    retriever = vectorstore.as_retriever()
    llm = ChatOpenAI(
        openai_api_base=BASE_URL,
        temperature=0,
        model="gpt-4o-2024-08-06"
    )
    system_prompt = (
        "Use the given context to answer the question. "
        "If you don't know the answer, say you don't know. "
        "Use three sentence maximum and keep the answer concise. "
        "Context: {context}"
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    qa_chain = create_stuff_documents_chain(llm, prompt)
    chain = create_retrieval_chain(retriever, qa_chain)

    def retrieval_tool(query):
        return chain.run({"input": query})

    return retrieval_tool


# 5. 创建增强版AssistantAgent
class PDFAssistant(AssistantAgent):
    def __init__(self, retrieval_tool):
        config_list = [{
            "model": "gpt-4o-2024-08-06",
            "base_url": BASE_URL,
            "api_type": "openai"
        }]
        
        super().__init__(
            name="pdf_assistant",
            llm_config={
                "config_list": config_list,
                "tools": [{
                    "type": "function",
                    "function": {
                        "name": "retrieve_tool",
                        "description": "从PDF文档检索信息",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "query": {"type": "string"}
                            },
                            "required": ["query"]
                        }
                    }
                }]
            }
        )
        
        self.register_function(function_map={"retrieve_tool": retrieval_tool})

    def query(self, question):
        response = self.generate_reply(
            messages=[{"content": question, "role": "user"}]
        )
        return response


# 主程序流程
if __name__ == "__main__":
    documents = load_and_process_pdf("/home/www/AgentFlow/agentflow/docs/GalsimToolkit.pdf")
    vectorstore = create_vector_store(documents)
    retrieval_tool = create_retrieval_tool(vectorstore)
    assistant = PDFAssistant(retrieval_tool)
    answer = assistant.query("请简要介绍Galsim")
    print("回答结果：\n", answer)

回答结果：
 Galsim是一个用于模拟天文学图像的开源软件库。它的全名是"Galaxies and Stars Image Simulation"。Galsim的主要功能包括：

1. **模拟天体图像**：用于模拟星系和恒星的图像，可以帮助天文学家在观测数据分析中进行校准和测试。

2. **多种模型支持**：支持各种星系和恒星的光学模型，比如Sérsic轮廓、光学器件的点扩散函数(PSF)模拟等。

3. **卷积和变形操作**：可以对图像进行卷积、变形（如剪切和放大）等操作，以精确模拟天体望远镜的观测效应。

4. **处理大规模仿真**：能够处理大规模的天体图像仿真，适用于应对大面积天空观测时的数据需求。

5. **灵活的用户接口**：提供了一个强大的Python接口，方便用户定制各种模拟场景。

Galsim在天文研究中非常有用，特别是在弱引力透镜、宇宙微波背景辐射等领域的模拟和分析中。
